In [1]:
import keras
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K



Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
from emnist import list_datasets
from emnist import extract_training_samples, extract_test_samples

print(list_datasets())
train_x, train_y = extract_training_samples('balanced')
test_x, test_y   = extract_test_samples('balanced')


# Get only the upper case letters
train_alphabet_list = (np.array(train_y) < 36) & (np.array(train_y) > 9)
test_alphabet_list  = (np.array(test_y) < 36) & (np.array(test_y) > 9)

train_y = train_y[train_alphabet_list] - 10
train_x = train_x[train_alphabet_list]
test_y = test_y[test_alphabet_list] - 10
test_x = test_x[test_alphabet_list]

train_x = train_x.reshape(train_x.shape[0], img_rows, img_cols, 1)
test_x = test_x.reshape(test_x.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

train_x = train_x.astype('float32')
test_x = test_x.astype('float32')
train_x /= 255
test_x /= 255

# convert class vectors to binary class matrices
train_y = keras.utils.to_categorical(train_y, 26)
test_y = keras.utils.to_categorical(test_y, 26)

print(train_x.shape)
print(test_x.shape)

print(train_y.shape)
print(test_y.shape)

['balanced', 'byclass', 'bymerge', 'digits', 'letters', 'mnist']
(62400, 28, 28, 1)
(10400, 28, 28, 1)
(62400, 26)
(10400, 26)


# Model

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(26, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


In [6]:
for i in range(20):
    print(i, end='\r')
    start = i * 50
    end   = start + 50
    model.fit(train_x[start:end], train_y[start:end],
              batch_size=25,
              epochs=20,
              verbose=0)


In [7]:
score = model.evaluate(test_x, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.4714755499362946
Test accuracy: 0.6699038461538461
